In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


학습 데이터를 만든다.

In [2]:
# xData => [공부시간, 과외시간]
temp = [[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]] # 7행 2열
print(temp)
print(type(temp))

xData = np.array(temp)
print(xData)
print(type(xData))

[[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
<class 'list'>
[[ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]]
<class 'numpy.ndarray'>


In [3]:
# yData => [합격여부] => 실제값
temp = [0, 0, 0, 1, 1, 1, 1] # 1행 7열
print(temp)
print(type(temp))

yData = np.array(temp)
print(yData)
print(type(yData))

# reshape(): numpy에서 데이터는 그대로 유지한채 배열의 형태(차원)을 변경한다.
# 1행 7열 numpy 배열 => 7행 1열 numpy 배열
yData = np.array(temp).reshape(7, 1)
print(yData)
print(type(yData))

[0, 0, 0, 1, 1, 1, 1]
<class 'list'>
[0 0 0 1 1 1 1]
<class 'numpy.ndarray'>
[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>


xData와 yData를 저장할 placeholder를 만든다.

In [4]:
# placeholder에 넘파이 배열을 대입하는 경우 shape 속성을 이용해서 대입되는 넘파이 배열의 차원을 지정해야 한다.
# [None, 2] => placeholder에 대입되는 넘파이 배열의 행의 개수는 몇개라도 상관없고 열의 개수는 무조건 2개라는 의미이다.
X = tf.placeholder(dtype=tf.float32, shape=[None, 2]) # xData가 대입될 placeholder
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1]) # yData가 대입될 placeholder

가중치와 a와 y절편 b를 임의의 값으로 정한다.

In [5]:
a = tf.Variable(tf.random_uniform([2, 1]), dtype=tf.float32) # 난수를 2행 1열로 발생시킨다.
b = tf.Variable(tf.random_uniform([1]), dtype=tf.float32)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('a1: {:f}, a2: {:f}, b: {:f}'.format(sess.run(a)[0][0], sess.run(a)[1][0], sess.run(b)[0]))
print('a\n', sess.run(a), sep='')
print('b\n', sess.run(b), sep='')

a1: 0.113474, a2: 0.521145, b: 0.698781
a
[[0.11347425]
 [0.52114487]]
b
[0.698781]


시그모이드 방정식, 오차 함수, 경사 하강법

In [6]:
# sigmoid(): 시그모이드 방정식을 계산한다.
# matmul(): 행렬의 곱(적)을 계산한다.
y = tf.sigmoid(tf.matmul(X, a) + b) # 시그모이드 방정식
loss = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y)) # 오차 함수
gradient_descent = tf.train.GradientDescentOptimizer(0.1).minimize(loss) # 경사 하강법

sigmoid() 함수의 실행 결과(y)가 0.5 이상이면 1을 0.5 미만이면 -1을 리턴시킨다.

In [7]:
sess = tf.Session()
# cast(x, dtype): x에 저장된 데이터를 dtype으로 형변환한다.
# predict = tf.cast(tf.constant([1.9, 2.1]), dtype=tf.int32)
# print(sess.run(predict)) # [1 2]
# cast() 함수는 캐스팅할 데이터가 boolean 타입이면 True는 1로 False는 0으로 형변환한다.
# predict = tf.cast([True, False], dtype=tf.int32)
# print(sess.run(predict)) # [1 0]
predict = tf.cast([0.4 >= 0.5, 0.5 >= 0.5], dtype=tf.int32)
print(sess.run(predict))

[0 1]


sigmoid() 함수를 실행한 예상값(0 또는 1)을 계산한다.

In [8]:
predict = tf.cast(y >= 0.5, dtype=tf.float32) # 예측값, sigmoid() 함수를 실행한 결과를 0 또는 1로 변환한다.

예측값(predict)과 실제값(Y)이 일치하는 정도(정확도, accuracy)를 계산한다.

In [9]:
sess = tf.Session()
# equal(): 인수로 지정한 값이 같으면 True, 다르면 False를 리턴한다.
# print(sess.run(tf.equal(1, 1)), sess.run(tf.equal(1, 0))) # True False
# equal() 함수로 예측값과 실제값이 같은가 비교 후 True 또는 False를 1 또는 0으로 캐스팅하고 전체 평균을 계산한다.
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), dtype=tf.int32))

학습시킨다.

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(3001):
    # sess.run([실행할내용, ...], feed_dict={X: 입력 데이터, Y: 입력 데이터에 따른 실제값})
    a_, b_, loss_, _ = sess.run([a, b, loss, gradient_descent], feed_dict={X: xData, Y: yData})
    if epoch % 300 == 0:
        print('epoch: {:5d}, a1: {:9.5f}, a2: {:9.5f}, b: {:9.5f}, loss: {:9.5f}'.format(epoch, a_[0][0], 
            a_[1][0], b_[0], loss_))

epoch:     0, a1:  -0.06776, a2:   0.26933, b:   0.22409, loss:   0.85528
epoch:   300, a1:   0.76389, a2:  -0.45522, b:  -2.57540, loss:   0.25794
epoch:   600, a1:   0.77620, a2:  -0.20656, b:  -4.01082, loss:   0.18600
epoch:   900, a1:   0.69648, a2:   0.10271, b:  -5.05275, loss:   0.14615
epoch:  1200, a1:   0.60079, a2:   0.39528, b:  -5.87949, loss:   0.12011
epoch:  1500, a1:   0.50795, a2:   0.65798, b:  -6.56677, loss:   0.10172
epoch:  1800, a1:   0.42312, a2:   0.89093, b:  -7.15552, loss:   0.08808
epoch:  2100, a1:   0.34720, a2:   1.09747, b:  -7.67076, loss:   0.07760
epoch:  2400, a1:   0.27967, a2:   1.28139, b:  -8.12896, loss:   0.06931
epoch:  2700, a1:   0.21957, a2:   1.44617, b:  -8.54160, loss:   0.06260
epoch:  3000, a1:   0.16591, a2:   1.59478, b:  -8.91697, loss:   0.05706


테스트

In [11]:
# 테스트 데이터를 만든다.
# new_x = np.array([6, 5]) # 테스트 데이터가 1차원 넘파이 배열로 만들어지기 때문에 행렬의 곱셈 연산이 실행되지 않는다.
# print(new_x) # [6 5] => 1차원
# reshape() 함수를 이용해서 1차원 배열이 아닌 1행 2열인 넘파이 배열로 테스트 데이터를 만들어야 한다.
new_x = np.array([1, 0]).reshape(1, 2)
# print(new_x) # [[6 5]] => 2차원

# sigmoid() 함수의 연산 결과에 테스트 데이터를 대입해서 연산한다.
result, new_y = sess.run([predict, y], feed_dict={X: new_x})
print('공부시간: %d, 과외시간: %d' % (new_x[:, 0], new_x[:, 1]))
print('합격여부: %s' % ('합격' if result == 1 else '불합격'))
print('합격확률: %f%%' % (new_y * 100))

공부시간: 1, 과외시간: 0
합격여부: 불합격
합격확률: 0.015828%


1시간도 공부하지 않고 과외수업만 받아서 합격할 합격정보

In [12]:
for i in range(11):
    new_x = np.array([0, i]).reshape(1, 2)
    result, new_y = sess.run([predict, y], feed_dict={X: new_x})
    print('공부시간: %d, 과외시간: %2d' % (new_x[:, 0], new_x[:, 1]), end=' ')
    print('합격여부: %s' % ('  합격' if result == 1 else '불합격'), end=' ')
    print('합격확률: %10.6f%%' % (new_y * 100))

공부시간: 0, 과외시간:  0 합격여부: 불합격 합격확률:   0.013414%
공부시간: 0, 과외시간:  1 합격여부: 불합격 합격확률:   0.066027%
공부시간: 0, 과외시간:  2 합격여부: 불합격 합격확률:   0.324497%
공부시간: 0, 과외시간:  3 합격여부: 불합격 합격확률:   1.578736%
공부시간: 0, 과외시간:  4 합격여부: 불합격 합격확률:   7.324693%
공부시간: 0, 과외시간:  5 합격여부: 불합격 합격확률:  28.028038%
공부시간: 0, 과외시간:  6 합격여부:   합격 합격확률:  65.739517%
공부시간: 0, 과외시간:  7 합격여부:   합격 합격확률:  90.434715%
공부시간: 0, 과외시간:  8 합격여부:   합격 합격확률:  97.898468%
공부시간: 0, 과외시간:  9 합격여부:   합격 합격확률:  99.566231%
공부시간: 0, 과외시간: 10 합격여부:   합격 합격확률:  99.911667%
